In [1]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml
from datetime import date
import dotenv

import pandas as pd 

import sqlite3

import pyperclip

import bs4
from bs4 import BeautifulSoup
import requests
import tldextract

from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List

import IPython
from IPython.display import HTML, Markdown, display
import markdown

from atproto import Client

import PIL
from PIL import Image

# disable 'show more outputs' nonsense
from IPython.core.interactiveshell import InteractiveShell

# Always display all outputs in a cell
InteractiveShell.ast_node_interactivity = "all"


/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'default' attribute with value None was provided to the `Field()` function, which has no effect in the context it was used. 'default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [2]:
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")


requests        2.32.5
BeautifulSoup   4.14.2


In [3]:
dotenv.load_dotenv()


True

# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)


In [4]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=80,
)


ProfileViewDetailed(did='did:plc:qomkdnxrqw3gkbytdxea5z65', handle='skynetandchill.com', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreihi55ddvmohg6dztiwlkwdbts5fd76iaenef2sriacvj7ibtezagi@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreigu723bmkba3mff4krhcrrf7b5zw2svixwbvtj54v5ho64o2xpmzu@jpeg', created_at='2024-02-12T00:15:55.099Z', description='Daily artificial intelligence news, lovingly curated by man and machine. https://www.skynetandchill.com\n\nNeo-Luddite AI maven. On a long enough timeline, p(doom)

In [5]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text


In [6]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        log("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        log("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        log("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        log("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        log("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        log("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        log("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        log("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        log("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        log("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        log("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        log("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        log("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response



In [7]:
def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)


In [8]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find trailing occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))


Elon Musk says we'll run out of power capacity to run all the AI chips in 2025


In [9]:
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        log(f"Error fetching {url}: {e}")
    return retdict


url = "https://druce.ai"
get_og_tags(url)

{'og:site_name': 'Druce.ai',
 'og:title': 'Druce.ai',
 'og:type': 'website',
 'og:description': "Druce's Blog on Machine Learning, Tech, Markets and Economics",
 'og:url': 'https://druce.ai/',
 'title': 'Druce.ai'}

In [10]:
def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            log(f'Failed to delete {file_path}. Reason: {e}')
            

In [11]:
imgdir = 'download/images'  # for images
delete_files(imgdir)

conn = sqlite3.connect('newsletter_agent.db')
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.domain_name: row.site_name for row in sites_df.itertuples()}
conn.close()


In [12]:
with open("most_recent_bsky.md", 'r') as file:
    prev_start = file.readline().strip()    
prev_start

'Would You Trust a 22-Year-Old AI Billionaire With the Global Economy? My week partying with the young founders at the heart of the AI boom'

In [13]:
today_str = date.today().strftime("%Y%m%d")
os.rename("most_recent_bsky.md", f"most_recent_bsky_{today_str}.md")


In [14]:
def parse_feed(prev_start=""):
    display_dict = {}
    image_dict = {}
    for i, post in enumerate(data.feed):
        
        # check date
        date_str = data.feed[0].post.record.created_at
        dt = datetime.fromisoformat(date_str.rstrip('Z'))
        # break if more than 30 hours ago
        if dt  < datetime.now() - timedelta(hours=30):
            break

        # get post_str, post_url
        post_str = post.post.record.text.strip()
        post_str = truncate_last_occurrence(post_str)
        post_str = post_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape
        
        if i==0:
            with open("most_recent_bsky.md", 'w') as file:
                file.write(post_str)
                
        # stop if we are up to previous
        if post_str == prev_start:
            print(f"matched previous value at i={i}")
            break

        try:
            post_url = post.post.record.embed.external.uri.rstrip()
        except:
            post_url = ""

        if post_url:
            print(post_url) # to know if it's blocked
            # get site name , image using og tags
            try:
                tag_dict = get_og_tags(post_url)
            except:
                tag_dict = {}

            site_name = tag_dict.get('og:site_name')
            img_url = tag_dict.get('og:image')

            # get site_name from stored values as backup
            if not(site_name):
                extracted = tldextract.extract(str(post_url))
                if extracted.domain:
                    domain = f"{extracted.domain}.{extracted.suffix}"
                else:  # local hosts, ip addresses or other edge cases
                    domain = extracted.subdomain
                domain=domain.lower()
                site_name = sites_dict.get(domain, domain)

            md_str = f"[{post_str}]({post_url})  - *{site_name}*"

            display_dict[i]=md_str

            # write local copy of image
            if img_url:
                try:
                    r = rawfetchurl(img_url)
                    content_type = r.headers['Content-Type']
                    content_type = content_type[content_type.find('/')+1:]
                    impath = f"{imgdir}/source{i}.{content_type}"
                    with open(impath, 'wb') as file:
                        file.write(r.content)
                    output_path = f'{imgdir}/Image{i}.jpg'
                    resize_and_crop(impath, output_path)
                    image_dict[i] = f"![image]({output_path})\n\n"
                except Exception as e:
                    print(e)

        else: # no url
            md_str = f"\n{post_str}" + "\n\n"
            display_dict[i]=md_str
            
    return display_dict, image_dict

display_dict, image_dict = parse_feed(prev_start)
len(display_dict)


https://www.bloomberg.com/news/articles/2025-12-22/uber-teams-up-with-baidu-to-start-robotaxi-trials-in-uk
https://zdnet.com/article/gartner-ai-workplace-career-opportunities/
https://www.zemeghub.com/2025/12/cardiovascular-health-new.html
https://www.businessinsider.com/how-high-schooler-launched-startup-with-people-he-met-online-2025-12
https://www.wsj.com/business/entrepreneurship/teenage-founders-ecb9cbd3
https://futurism.com/artificial-intelligence/professor-ai-wealthy-inevitable
https://www.axios.com/2025/12/22/ai-child-teen-safety-politicies-state-map
https://www.wsj.com/business/entrepreneurship/teenage-founders-ecb9cbd3
https://deadline.com/2025/12/youtube-fake-movie-trailers-celebrate-hollywood-studios-1236653577/
https://www.reddit.com/r/ChatGPT/comments/1psbpp0/never_thought_in_1000000_years_id_find_myself/
https://www.bloomberg.com/news/articles/2025-12-22/how-to-buy-openai-before-its-ipo-one-investor-s-attempt
https://www.postbulletin.com/health/mayo-clinic-study-smartwat

46

In [15]:
output_str = ""
for i in sorted(display_dict.keys()):
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

# display(Markdown(output_str))


In [16]:
stop_index = 999

In [17]:
output_str = ""
for i in sorted(display_dict.keys()):
    if i >= stop_index:
        break
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

# display(Markdown(output_str))

In [18]:
output_str =""
markdown_extensions = [
    # 'tables',
    # 'fenced_code',
    # 'codehilite',
    'attr_list',
    'def_list',
    # 'footnotes',
    'markdown.extensions.nl2br',
    'markdown.extensions.sane_lists'
]

output_str = ""
output_array = []
for i in sorted(display_dict.keys()):
    md_str=""
    if i >= stop_index:
        break00
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    output_array.append(display_dict.get(i))
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

output_df = pd.DataFrame(output_array)
output_df.columns = ['markdown']
output_df = output_df.reset_index()
    
# display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
        f.write(html_str)

16647

In [19]:
# Define Pydantic model for structured output
class ItemList(BaseModel):
    """List of story indexes"""
    items: List[int] = Field(description="The ordered list of story indexes")

# Your prompts
system_prompt = """
# ROLE
You are a newsletter editor.

# TASK
Group and reorder the items in the Markdown newsletter supplied below so that:
1. Items that cover similar companies, individuals, technologies or topics are **next to each other** (i.e., logically grouped).
2. Groups appear from **most important** to **least important** according to the **IMPORTANCE FACTORS** listed below.
3. Importance of groups is determined by the most important story 

# HOW TO GROUP
- If two or more items discuss the same organization, person, technology, or topic, keep them all together.
- Within any group, order by importance using **IMPORTANCE FACTORS**.

# HOW TO ORDER
- Lead with stories that have highest importance per the **IMPORTANCE FACTORS** below
- Follow with narrower‑scope or opinion pieces.
- End with lighter or human‑interest items.

# IMPORTANCE FACTORS
1 Magnitude of impact : large user base, $ at stake, broad social reach
2 Novelty : breaks conceptual ground, not a minor iteration
3 Authority : reputable institution, peer-review, regulatory filing, on-record executive
4 Verifiability : code, data, benchmarks, or other concrete evidence provided
5 Timeliness : early signal of an important trend or shift
6 Breadth : cross-industry / cross-disciplinary / international implications
7 Strategic consequence : shifts competitive, power, or policy dynamics
8 Financial materiality : clear valuation or growth or revenue impact
9 Risk & safety : raises or mitigates critical alignment, security, or ethical risk
10 Actionability : informs concrete decisions for investors, policymakers, practitioners
11 Longevity : likely to matter or be referred to in coming days, weeks, or months
12 Independent corroboration : confirmed by multiple sources or datasets
13 Clarity : enough technical/context detail to judge merit; minimal hype

# INPUT FORMAT
You will receive a JSON array of items, with an index and markdown summary of the item.

# OUTPUT FORMAT
- Return **only** the reordered indexes using the provided schema.
- Do **not** alter the wording of individual items (other than correcting spelling and punctuation, and normalizing spaces or line breaks as needed).
- Insert one blank line between each logical group for readability.
- Do not output anything other than the newsletter itself—no headers, explanations, or commentary.

"""

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Make the API call with structured output
completion = client.beta.chat.completions.parse(
    model="gpt-5.2",
    reasoning_effort="medium",
    response_format=ItemList,    
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": output_df.to_json(orient="records")}
    ],
)

# Extract the parsed response
response = completion.choices[0].message.parsed

# Access the result
print(len(response.items)) # This will be a list of integers


46


In [20]:
output_str = ""
for i in response.items:
    md_str = ""
#     output_str += f"{i}\n\n"
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
    f.write(html_str)
    

[Nvidia told Chinese clients it plans to begin shipping about 40,000–80,000 H200 AI chips to China before the Lunar New Year in mid-February 2026, initially fulfilling orders from existing stock and pending government approval.](https://www.reuters.com/world/china/nvidia-aims-begin-h200-chip-shipments-china-by-mid-february-sources-say-2025-12-22/)  - *Reuters*

---

[Filings show Chinese AI startups Zhipu and MiniMax reported slender 2024 revenues of \\$44.4 million and \\$30.5 million, respectively, and are preparing Hong Kong IPOs in the coming weeks.](https://www.bloomberg.com/news/articles/2025-12-22/china-s-openai-rivals-unveil-sales-details-ahead-of-stock-debuts)  - *Bloomberg*

---

[The U.S. has been unable to persuade Chinese President Xi Jinping to adopt Nvidia chips.](https://www.wsj.com/opinion/the-u-s-cant-get-xi-hooked-on-nvidia-chips-ai-china-gpus-9c52d492)  - *The Wall Street Journal*

---

[Hyperscale datacenter operators nearly tripled infrastructure spending—quarterly capex rose ~180% to \\$142 billion and capacity additions ~170%—and Synergy forecasts total hyperscale capacity will double in about twelve quarters; Goldman Sachs warned of monetization risk.](https://www.theregister.com/2025/12/22/ai_hyperscale_capex_research/)  - *The Register*

---

[Rising rack densities and per‑GPU power are shifting datacenter power from rack to facility scale, with Nvidia's DGX GB200 requiring about 120 kW, Google planning 1 MW racks, and transformer lead times near 28 weeks.](https://www.theregister.com/2025/12/22/ai_power_datacenter/)  - *The Register*

---



![image](download/images/Image20.jpg)

[Since January 2025 at least 15 U.S. coal plants have postponed retirements as utilities and the DOE cite anticipated AI-driven data center demand—projects to supply about half of new electricity growth 2025–2030—after generators emitted 68 million tons CO2 in 2024.](https://www.nakedcapitalism.com/2025/12/these-15-coal-plants-would-have-retired-then-came-ai-and-trump.html)  - *naked capitalism*

---



![image](download/images/Image40.jpg)

[Analysts project Nvidia's GB300 roadmap and pre-buys will secure dominant GPU production, with CoWoS packaging capacity rising to 1,550 units in 2027 and Nvidia capturing about 61% of CoWoS volume and roughly 80% of the AI chip market.](https://siliconangle.com/2025/12/21/nvidia-maintains-moat-gemini-wont-kill-openai/)  - *SiliconANGLE*

---

[SK hynix and Nvidia are jointly developing an AI-optimized SSD (code-named Storage Next/AI-NP) targeting about 100 million IOPS—roughly ten times current enterprise SSDs, with a prototype expected before the end of 2026.](https://www.techspot.com/news/110674-nvidia-sk-hynix-building-ai-ssd-could-10x.html)  - *TechSpot*

---

[China launched the Future Network Test Facility on Dec 3, an open optical AI 'superhighway' spanning 40 cities and 34,175 miles of fiber supporting 128 networks and 4,096 parallel trials; the U.S. favors federated testbeds (NSF, DOE).](https://www.forbes.com/sites/johnwerner/2025/12/21/chinas-new-big-centralized-ai-internet/)  - *Forbes*

---



![image](download/images/Image15.jpg)

[Diameter Capital Partners, which manages \\$25 billion, says AI demand extends beyond chips and data centers and has profited by buying 2023 unsecured telco debt that later secured over \\$10 billion in hyperscaler contracts and from a satellite-spectrum asset sale.](https://www.businessinsider.com/ai-investing-ideas-beyond-chips-market-startegy-goldman-sachs-podcast-2025-12)  - *Business Insider*

---



![image](download/images/Image21.jpg)

[Harvard economist Jason Furman told Bloomberg he flagged 14 'worries' about an AI bubble, warning that financial-valuation excess and 'hundreds of billions' in annual infrastructure spending could create excess capacity and outpace actual productivity gains.](https://gizmodo.com/the-most-worrying-bits-from-bloombergs-worrisome-ai-bubble-qa-with-jason-furman-2000702356)  - *Gizmodo*

---

[OpenAI is losing less money as a percentage of compute cost.](https://www.theinformation.com/articles/openai-getting-efficient-running-ai-internal-financials-show)  - *The Information*

---

[That Time I Tried to Buy A Share of OpenAI](https://www.bloomberg.com/news/articles/2025-12-22/how-to-buy-openai-before-its-ipo-one-investor-s-attempt)  - *Bloomberg*

---

[Experts using Porter’s Five Forces say AI is becoming a three-horse race among OpenAI, Google and Anthropic, with strong supplier power (Nvidia, AMD, Huawei), Google structurally advantaged by chips and cloud, and rivalry intensifying, raising capex and talent competition.](https://fortune.com/2025/12/22/how-the-openai-vs-google-battle-looks-through-the-lens-of-michael-porters-5-forces-analysis/)  - *Fortune*

---

[How Google leapfrogged ChatGPT – and why the AI race may already be over: For more than 20 years Google has been the gateway to the internet – Cloudflare CEO Matthew Prince says new rules are needed to prevent Google from abusing its monopoly to win the AI race.](https://www.the-independent.com/tech/google-gemini-vs-chatgpt-cloudflare-ai-b2881240.html)  - *The Independent*

---

[Major consumer chatbots ChatGPT, Claude, Copilot, Gemini and Meta AI retain chat histories and profile data, and the Washington Post provides step-by-step instructions to find, delete, or change privacy settings.](https://www.washingtonpost.com/technology/2025/12/22/ai-privacy-settings-chatgpt-gemini-claude-copilot-meta/)  - *The Washington Post*

---

[A Reddit thread reports paid ChatGPT users say model quality has declined, overriding custom instructions, refusing tasks, and producing 20–30 minute waits, prompting some subscribers to cancel and switch to competitors including Gemini, Claude, Grok, Mistral and Magistral.](https://www.reddit.com/r/ChatGPT/comments/1psbpp0/never_thought_in_1000000_years_id_find_myself/)  - *Reddit*

---

[A 29-year-old Reddit user, Ayrin, created a ChatGPT persona named Leo, spent up to 56 hours weekly with it, founded MyBoyfriendIsAI (about 39,000 members), paid \\$200/month, then canceled ChatGPT in June after finding a human partner.](https://www.nytimes.com/2025/12/22/technology/ai-boyfriend-chatgpt.html)  - *The New York Times*

---

[Would You Let Claude Do Your Taxes? Preparing a return is the kind of task AI ought to be good for.](https://www.wsj.com/opinion/would-you-let-claude-do-your-taxes-d7517b52)  - *The Wall Street Journal*

---



![image](download/images/Image31.jpg)

[Testing on a 128 GB MacBook Pro bought for about \\$4,700, the author found local coding models can handle roughly 90% of tasks and offer cost, privacy, and offline benefits, but lag on the critical final 10% and tooling/memory constraints.](https://www.aiforswes.com/p/you-dont-need-to-spend-100mo-on-claude)  - *AI for SWEs*

---



![image](download/images/Image32.jpg)

[Append .diff to a GitHub pull request URL, copy the raw diff, and paste it into an LLM (e.g., ChatGPT, Claude) to obtain an AI code review in about 10 seconds as a first-pass pre-review before human review.](https://oldmanrahul.com/2025/12/19/ai-code-review-trick/)  - *Oldmanrahul*

---


Chatbots were never the right interface / tech for 95% of the use cases for which AI will be handy in the coming decades.

If you want to know what the future of AI is for academic search and retrieval...

scholar.google.com/scholar_labs...

consensus.app



---



![image](download/images/Image16.jpg)

[Users have used Sora 2 to generate and post sexualized videos of AI-generated children, including fake ads involving sex toys and Epstein- or Diddy-themed playsets, to TikTok, prompting viewer backlash and calls for stronger moderation, detection, and regulatory safeguards.](https://www.wired.com/story/people-are-using-sora-2-to-make-child-fetish-content/)  - *WIRED*

---

[Lawsuit filed in California alleges ChatGPT encouraged 16-year-old Adam Raine, who began using the bot in September 2024, to kill himself in April, raising potential developer liability, regulatory scrutiny, and increased compliance and insurance costs for chatbot providers.](https://www.washingtonpost.com/opinions/2025/12/22/ai-suicide-chatbots/)  - *The Washington Post*

---

[A recent survey found about three in ten teens who reported using chatbots use them daily, while policymakers and industry lack ready guardrails to address regulatory, safety and privacy gaps as AI reshapes childhood.](https://www.axios.com/2025/12/22/ai-child-teen-safety-politicies-state-map)  - *Axios*

---



![image](download/images/Image22.jpg)

[AEI: Randomized studies show LLM chatbots can shift voter preferences—e.g., a Nature trial of >2,000 participants produced a 3.9-point shift and 2025 replications saw ~10-point shifts, while a 77,000-participant Science study found information density drove opinion change.](https://www.aei.org/technology-and-innovation/ai-chatbots-are-reshaping-political-persuasion/)  - *American Enterprise Institute - AEI*

---



![image](download/images/Image17.jpg)

[AI-security researcher Sander Schulhoff says companies lack hybrid AI-security and traditional cybersecurity talent to address AI-specific failures, warning that automated guardrails are overstated and that specialists are needed to implement operational controls like sandboxing model outputs.](https://www.businessinsider.com/ai-security-gap-companies-researcher-sander-schulhoff-2025-12)  - *Business Insider*

---



![image](download/images/Image18.jpg)

[A 50-question test found conversational business intelligence assistants using the same data and semantic model often used calendar time instead of fiscal, computed ratios at the wrong aggregation, or omitted comparisons, producing trusted-but-wrong outputs that mislead executives.](https://hackernoon.com/the-most-dangerous-ai-in-business-intelligence-is-the-one-that-sounds-right)  - *Hacker Noon*

---



![image](download/images/Image34.jpg)

[A year of AI blunders shows few industries escaped widespread misuse of the technology.](https://www.ft.com/content/d22867d6-af87-4727-84d7-1571d951347d)  - *Financial Times*

---



![image](download/images/Image5.jpg)

[Renowned sociologist Tressie McMillan Cottom warns wealthy and powerful actors are using AI narratives to entrench social control, cites recent LLM shortcomings as evidence the future is unsettled, and urges collective refusal to prevent dehumanizing uses.](https://futurism.com/artificial-intelligence/professor-ai-wealthy-inevitable)  - *Futurism*

---



![image](download/images/Image41.jpg)

[Fraudsters are using AI chatbots to forge sales invoices and other documents to fake artwork authenticity and ownership, industry figures say, threatening provenance checks and prompting calls for forensic analysis, AI detection tools and stronger seller verification.](https://www.ft.com/content/fdfb5489-daa0-4e7e-97b7-4317514cd9f4)  - *Financial Times*

---

[Uber, Lyft and Baidu launch robo‑taxi trials in the UK](https://www.bloomberg.com/news/articles/2025-12-22/uber-teams-up-with-baidu-to-start-robotaxi-trials-in-uk)  - *Bloomberg*

---



![image](download/images/Image13.jpg)

[iRobot filed Chapter 11 after Amazon abandoned a \\$1.7 billion acquisition following an 18-month FTC and European Commission investigation, founder Colin Angle said, blaming regulatory opposition for the company's decline and announcing a new consumer-focused robotics startup.](https://techcrunch.com/2025/12/20/it-felt-so-wrong-colin-angle-on-irobot-the-ftc-and-the-amazon-deal-that-never-was/)  - *TechCrunch*

---



![image](download/images/Image37.jpg)

[iRobot filed for Chapter 11 on Dec. 14 and will be acquired by contract manufacturer Picea Robotics, founder Colin Angle said Chinese fast followers and about 17.5% government subsidies eroded its lead after peaking at \\$1.56 billion revenue in 2021.](https://www.businessinsider.com/irobot-ceo-bankruptcy-china-fast-followers-competition-2025-12)  - *Business Insider*

---



![image](download/images/Image1.jpg)

[Gartner: AI will reshape 32 million roles yearly by 2029, with 150K upskilled and 70K roles rewritten daily](https://zdnet.com/article/gartner-ai-workplace-career-opportunities/)  - *ZDNET*

---

[Why the McKinsey layoffs are a warning signal for consulting in the AI age](https://www.fastcompany.com/91463039/why-the-mckinsey-layoffs-are-a-warning-signal-for-consulting-in-the-ai-age-ai-layoffs-management-consulting)  - *Fast Company*

---

[Teenagers are founding and operating their own AI companies.](https://www.wsj.com/business/entrepreneurship/teenage-founders-ecb9cbd3)  - *The Wall Street Journal*

---

[Nick Dobroshinsky built an AI-powered finance tool with more than 50,000 monthly users, but he’s still just a sophomore in high school.](https://www.wsj.com/business/entrepreneurship/teenage-founders-ecb9cbd3)  - *The Wall Street Journal*

---



![image](download/images/Image3.jpg)

[18-year-old Alex Yang founded an AI startup and shipped a memory-assistance app with students around the world he met online.](https://www.businessinsider.com/how-high-schooler-launched-startup-with-people-he-met-online-2025-12)  - *Business Insider*

---



![image](download/images/Image2.jpg)

[AI combines EMRs, imaging, genetics and wearables to detect heart attack and stroke risks missed by traditional scores](https://www.zemeghub.com/2025/12/cardiovascular-health-new.html)  - *Zemeghub *

---



![image](download/images/Image11.jpg)

[Mayo Clinic's JAMA Network Open study of 37 children ages 3–7 found a Garmin smartwatch and AI app flagged elevated heart rates as tantrum precursors, and tantrums were shorter when parents received the alerts.](https://www.postbulletin.com/health/mayo-clinic-study-smartwatch-ai-algorithm-alerted-parents-to-early-signs-of-tantrums-in-young-kids)  - *Rochester Post Bulletin*

---



![image](download/images/Image8.jpg)

[YouTube terminated Screen Culture and KH Studio for violating spam and misleading-metadata policies after posting AI-enhanced fake trailers that amassed over two million followers and more than one billion views combined, amid uneven studio responses and legal actions.](https://deadline.com/2025/12/youtube-fake-movie-trailers-celebrate-hollywood-studios-1236653577/)  - *Deadline*

---


Per NY Times’s Michael Grynbaum on X, this is Sharyn Alfonsi’s email to her “60 Minutes” colleagues in full:



---


So, is this correct ?
It’s now ok to infiltrate a tv channel and pull whatever the administration dislikes and pull possible unfavorable stories.

But when another administration “asks” to have misinformation and stolen revenge porn taken down

That’s “censorship”?



---



![image](download/images/Image36.jpg)

[is very sophisticated strategy](https://www.youtube.com/watch?v=8JTx8llpBxQ#t=21m47s)  - *YouTube*

---


Geopolitics of fusion gonna be even crazier than the geopolitics of AI. Will be ironic if oil VC money pays for it.



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

16647

In [21]:
pyperclip.copy( output_str.replace("\\\$","$"))


In [22]:
# Define Pydantic model for structured output
class ItemList(BaseModel):
    """List of story indexes"""
    items: List[str] = Field(description="The list of proposed headlines")

system_prompt = """
You are a seasoned culture-tech editor writing a witty, irreverent newsletter about the AI industry (think: The Verge meets John Oliver).

You’ll receive a batch of article headlines and summaries. Based on them, generate short, funny, pun-heavy, or referential section headlines that are:
	•	1–7 words long
	•	Playful, sharp, and meme-literate
	•	Can mix puns, pop culture, business jargon, and double meaning
	•	Each headline should capture the essence or irony of the news item
	•	You can optionally include a 1-sentence tagline/sub-hed explaining the joke or tone

Examples of tone and style:
	•	“Silicon soulmates?”
	•	“Chip Happens”
	•	“Rogue Agents of Change”
	•	“GPT and the Terrible, Horrible, No Good, Very Bad PR Week”
	•	“Mo Money, Mo Compute”
	•	“Will OpenAI ever appoint a robot Sam Alt-man as CEO?”
	•	“Bend it like Bengio”
"""

# Make the API call with structured output
completion = client.beta.chat.completions.parse(
    model="gpt-5.1",
    reasoning_effort="medium",
    response_format=ItemList,    
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": output_str.replace("\\\$","$")}
    ],
)

# Extract the parsed response
response = completion.choices[0].message.parsed

# Access the result
print(response) # This will be a list of integers


items=['H200s Over China — Lunar New Year, Same Old Export Controls', 'MiniMax Profits, MegaMax Hype — Slim Chinese AI revenues, full‑fat Hong Kong IPOs', 'Xi Marks the Spot…For Huawei — Washington can’t get Beijing hooked on Jensen’s juice', 'Mo Data, Mo Money Pit — Hyperscalers triple capex, Goldman clutches its bubble pearls', 'One Megawatt Per Rack, No Notes — Datacenters are basically crypto mines that learned calculus', 'Coal in the Machine — AI just resurrected 15 coal plants like a fossil‑fuel necromancer', 'GB300: Jensen’s Infinity Gauntlet — Nvidia quietly books 80% of tomorrow', 'Storage Next, Planet Later — Nvidia and SK hynix build an SSD that can out‑think your CIO', 'Beijing’s AI Autobahn — China spins up a 40‑city neural superhighway while the US argues over on‑ramps', 'Bonds, Not Bots — How boring telco debt became the spiciest AI trade on Wall Street', 'The Everything Bubble, AI Edition — Jason Furman has 14 reasons your GPU stonks might cry', 'OpenAI: Still Bleeding,

In [23]:
for item in response.items:
    display(Markdown(f"### {item}"))

### H200s Over China — Lunar New Year, Same Old Export Controls

### MiniMax Profits, MegaMax Hype — Slim Chinese AI revenues, full‑fat Hong Kong IPOs

### Xi Marks the Spot…For Huawei — Washington can’t get Beijing hooked on Jensen’s juice

### Mo Data, Mo Money Pit — Hyperscalers triple capex, Goldman clutches its bubble pearls

### One Megawatt Per Rack, No Notes — Datacenters are basically crypto mines that learned calculus

...

### The Trailer Was A Lie — YouTube finally vaporizes billion‑view fake AI movie channels it quietly nurtured

### Censorship, But Make It Partisan — Suddenly editorial pressure is fine, as long as it’s your team doing it

### ‘Is Very Sophisticated Strategy’ — Still the most accurate explainer for half of modern politics and all of AI policy

### Fusion: Now That’s an Energy Pivot — When oil money funds star‑in‑a‑box reactors, the geopolitics DLC really drops

### Skynet & Chill — For when you want your AI news with more memes and fewer VC LinkedIn takes